In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from store import *
import kbech32

In [2]:
store = Store(os.getenv('localappdata') + r'\Kaspad\kaspa-mainnet\datadir2')
store.load_blocks()

Pruning point:  bb7cdefbb7bd87a2958ee366266658a2911026d188e2d484cdb7b392c2579ff6
Loaded 40000 blocks
Loaded 80000 blocks
Loaded 120000 blocks
Loaded 160000 blocks
Loaded 200000 blocks
Loaded 240000 blocks
Overall number of headers:  244752


In [3]:
header_fields = ['timeInMilliseconds', 'blueScore', 'blueWork', 'daaScore', 'difficulty']
block_fields = ['pubkey_script', 'kaspad_version', 'miner_version']
count_fields = [] # ['num_parents', 'num_children', 'num_blues', 'num_reds']
frames = store.load_data(header_fields=header_fields, block_fields=block_fields, count_fields=count_fields)

  0%|          | 0/244752 [00:00<?, ?it/s]

Number of headers missing header data:  1
Number of blocks missing block data:  1


In [4]:
frames.keys()

dict_keys(['hash', 'timeInMilliseconds', 'blueScore', 'blueWork', 'daaScore', 'difficulty', 'pubkey_script', 'kaspad_version', 'miner_version'])

In [5]:
df = pd.DataFrame(frames).set_index('hash')

In [6]:
pp_time = store.get_header_data(store.pruning_point()).timeInMilliseconds
pp_time

1650151162040

In [7]:
df["address"] = df["pubkey_script"].apply(kbech32.toAddress)

In [8]:
df['kaspad_version'].value_counts()

0.11.17    143391
0.11.14     63442
0.12.0      29299
0.11.15      7505
0.11.16       626
unknown       480
0.11.13         8
Name: kaspad_version, dtype: int64

In [9]:
time_filter = df['timeInMilliseconds'] > pp_time + 160000*1000
np.sum(time_filter)

82204

In [10]:
df[time_filter]['kaspad_version'].value_counts()

0.11.17    69422
0.12.0     11265
0.11.15      982
0.11.16      221
unknown      163
0.11.14      150
0.11.13        1
Name: kaspad_version, dtype: int64

In [11]:
df[time_filter]['miner_version'].value_counts()

                                43007
eu2.acc-pool.pw                 27032
ge.acc-pool.pw                   5835
BzMiner-v8.1.4                   1932
ca.acc-pool.pw                   1907
0.2.1-GPU-0.5/2204021842          755
ru.acc-pool.pw                    495
unknown                           314
hk.acc-pool.pw                    286
0.2.1-GPU-0.5/2204021837          238
BzMiner-v8.2.0b1                  125
0.2.1-GPU-0.5/2204081349          122
0.2.1-GPU-0.5/2204070122           49
0.2.1-GPU-0.5-rc1/2204092029       44
0.2.1-GPU-0.5/2204081340           17
0.2.1-GPU-0.5/2204061443           16
0.2.1-GPU-0.6/2204181211           13
0.2.1-GPU-0.6/2204121204            9
0.2.1-GPU-0.5/2204070121            8
Name: miner_version, dtype: int64

In [13]:
(df[time_filter]['kaspad_version'] + '/' + df[time_filter]['miner_version']).value_counts()

0.11.17/                               41914
0.11.17/eu2.acc-pool.pw                27032
0.12.0/ge.acc-pool.pw                   5835
0.12.0/BzMiner-v8.1.4                   1587
0.12.0/ca.acc-pool.pw                   1134
0.12.0/                                 1093
0.11.15/ca.acc-pool.pw                   773
0.12.0/0.2.1-GPU-0.5/2204021842          538
0.12.0/ru.acc-pool.pw                    495
0.11.17/hk.acc-pool.pw                   286
0.11.16/0.2.1-GPU-0.5/2204021842         211
0.11.15/BzMiner-v8.1.4                   203
0.12.0/0.2.1-GPU-0.5/2204021837          196
unknown/unknown                          163
0.11.14/unknown                          150
0.11.17/BzMiner-v8.1.4                   132
0.12.0/0.2.1-GPU-0.5/2204081349          121
0.12.0/BzMiner-v8.2.0b1                  110
0.12.0/0.2.1-GPU-0.5/2204070122           49
0.12.0/0.2.1-GPU-0.5-rc1/2204092029       44
0.11.17/0.2.1-GPU-0.5/2204021837          42
0.12.0/0.2.1-GPU-0.5/2204081340           17
0.12.0/0.2

In [14]:
# store.close()